# Práctico 2a: Neurona de Wilson

## Configuración

Ejecutá las siguientes celdas para configurar el entorno del notebook

In [ ]:
import numpy as np
import matplotlib.pylab as plt
import ipywidgets as widgets

### Funciones utilitarias

In [ ]:
def I_inj(t, amp):
  """Crea una función de corriente inyectada con un pulsos cuadrado.

  Args:
    t (ndarray): vector 1D de tiempos (ms), estrictamente creciente.
    amp (float): amplitud del pulso, en uA/cm^2.

  Returns:
    function: una función I(t) que devuelve la corriente inyectada I(t).
  """
  tmax = np.max(t)
  inicios = np.linspace(0, tmax, 8)
  inicio = inicios[1]
  fin = inicios[6]
  
  def I(t):
    return amp * (t > inicio) - amp * (t > fin)

  return I

### Funciones de graficado

In [ ]:
def plot_wilson(t, I, V, R, T, H):
  """Grafica la dinámica del modelo de Wilson.

  Args:
    t (ndarray): vector 1D de tiempos (ms), estrictamente creciente y con paso uniforme.
    I (function): función escalar I(t) que devuelve la corriente externa (mA) en cada t.
    V (ndarray): potencial de membrana.
    R (ndarray): variable de compuerta R.
    T (ndarray): variable de compuerta T.
    H (ndarray): variable lenta H.
  """
  fig, [ax1, ax2, ax3, ax4, ax5] = plt.subplots(5, 1, figsize=(8, 7), gridspec_kw={"height_ratios": [3, 2, 2, 2, 1]}, sharex=True)

  ax1.plot(t, V * 100)
  ax1.set_ylim(-100, 50)
  ax1.set_ylabel("V (mV)")
  ax1.grid(True)
  
  ax2.plot(t, R * 100)
  ax2.set_ylim(0, 250)
  ax2.set_ylabel("R")
  ax2.grid(True)
  
  ax3.plot(t, T * 100)
  ax3.set_ylim(0, 250)
  ax3.set_ylabel("T")
  ax3.grid(True)
  
  ax4.plot(t, H * 100)
  ax4.set_ylim(0, 250)
  ax4.set_ylabel("H")
  ax4.grid(True)
  
  ax5.plot(t, I(t))
  ax5.set_ylim(-2, 2)  
  ax5.set_ylabel("I (mA)")
  ax5.set_xlabel("t (ms)")
  
  plt.show()

## Definición del modelo

Wilson (1999) simplificó el modelo de Hodgkin-Huxley con dos ecuaciones diferenciales:

$$C \dfrac{dV}{dt} = -g_K R (V - E_K) - g_{Na}(V) (V - E_{Na}) + I(t)$$

$$\tau_R \dfrac{dR}{dt} = -[R - R_0(V)]$$

Estas ecuaciones aproximan a los mecanismos principales descriptos por Hodgkin y Huxley. Sin embargo, en las neuronas corticales encontramos dos nuevos tipos de canales iónicos esenciales para describir propiedades de disparo diferentes. Para incorporar estos canales, el modelo de Wilson incorpora dos variables dinámicas: $H$ y $T$, por lo que el modelo completo de neuronas corticales de Wilson esta dado por:

$$C \dfrac{dV}{dt} = -g_K R (V - E_K) - g_{Na}(V) (V - E_{Na})- g_T T (V - E_T)- g_H H (V - E_H) + I(t)$$
$$\tau_R \dfrac{dR}{dt} = -[R - R_0(V)]$$
$$\tau_T \dfrac{dT}{dt} = -[T - T_0(V)]$$
$$\tau_H \dfrac{dH}{dt} = -[H - 3 T(V)]$$

con parametrizaciones polinómicas de la dependencia de voltaje de las conductancias efectivas:

$$g_{Na} = 17.8 + 47.6 * V + 33.8 * V^2$$
$$R_0 = 1.24 + 3.7 * V + 3.2 * V^2$$
$$T_0 = 4.205 + 11.6 * V + 8 * V^2$$

A pesar de estas simplificaciones, este modelo aproxima a varios patrones de disparo observados en neuronas corticales con gran detalle.

Veamos como implementarlo en Python.

### Constantes

In [ ]:
# Condición inicial de la variable de voltaje V(t) en t=0, es decir, valor de V(0), en mV/100.
V_0 = np.float64(-0.75)

# Condición inicial de la variable de recobro R(t) en t=0, es decir, valor de R(0), en mV/100.
R_0 = np.float64(0.26)

# Condición inicial de las variables de conductancia T(t) y H(t) en t=0, es decir, valor de R(0), en mV/100.
T_0 = np.float64(0)
H_0 = np.float64(0)

# Potenciales de equilibrio de K, Na, T y H, en mV/100
E_K = -.95
E_Na = .50
E_T = 1.20
E_H = -.95

# Capacitancia de la membrana, en uFcm^-2
C = 1

*Nota: El potencial de membrana esta escalado en unidades de $mV/100$ para mantener las constantes de las ecuaciones de $g_{Na}$ y $R_0$ en un rango razonable. Al graficarla multiplicaremos por un factor de 100 para compararlos con datos fisiológicos.*

### Integración

Para simular tenemos que integrar las ecuaciones diferenciales $\dfrac{dV}{dt}$, $\dfrac{dR}{dt}$, $\dfrac{dT}{dt}$ y $\dfrac{dH}{dt}$ a lo largo del tiempo $t$. Para eso, vamos a utilizar el método de Euler según vimos en el Cuaderno 3.

In [ ]:
def vdot(V, R, T, H, g_K, g_T, g_H, I):
    g_Na = 17.8 + 47.6 * V + 33.8 * np.power(V, 2)
    I_K = g_K * R * (V - E_K)
    I_Na = g_Na * (V - E_Na)
    I_T = g_T * T * (V - E_T)
    I_H = g_H * H * (V - E_H)
    return (-I_K - I_Na - I_T - I_H + I) / C
    
def rdot(R, tau_R, V):
    R_0 = 1.24 + 3.7 * V + 3.2 * np.power(V, 2)
    return -(R - R_0) / tau_R

def tdot(T, tau_T, V):
    T_0 = 4.205 + 11.6 * V + 8 * np.power(V, 2)
    return -(T - T_0) / tau_T
    
def hdot(H, tau_H, T):
    return -(H - 3 * T) / tau_H

def integrar_wilson(t, I, tau_R=4.5, tau_T=14, tau_H=45, g_K=26, g_T=2.25, g_H=9.5):
  """Integra el sistema de Wilson (V, R, T, H) con Euler.

  Args:
    t (ndarray): vector 1D de tiempos (ms), estrictamente creciente y con paso uniforme.
    I (function): función escalar I(t) que devuelve la corriente externa en ms.
    tau_R (float): constante de tiempo de R (ms).
    tau_T (float): constante de tiempo de T (ms).
    tau_H (float): constante de tiempo de H (ms).
    g_K (float): conductancia del canal K.
    g_T (float): conductancia del canal T.
    g_H (float): conductancia del canal H.

  Returns:
    tuple: (V, R, T, H), cada uno como ndarray con la evolución temporal en los tiempos t.
  """
  
  # Variables a integrar
  V = np.array([V_0])
  R = np.array([R_0])
  T = np.array([T_0])
  H = np.array([H_0])

  # Calculamos el valor de salto en el tiempo
  dt = t[1] - t[0]
  
  # Integramos con el método de Euler
  for step in t[:-1]:
    V = np.append(V, V[-1] + dt * vdot(V[-1], R[-1], T[-1], H[-1], g_K, g_T, g_H, I(step)))
    R = np.append(R, R[-1] + dt * rdot(R[-1], tau_R, V[-2]))
    T = np.append(T, T[-1] + dt * tdot(T[-1], tau_T, V[-2]))
    H = np.append(H, H[-1] + dt * hdot(H[-1], tau_H, T[-2]))

  return V, R, T, H

*Nota: El modelo de Wilson incorpora una variable $T$ que es muy similar a la variable que usamos para el tiempo $t$, tener en cuenta esta diferencia al leer el código de Python.*

## Simulación del modelo

Ahora sí, podemos simular el modelo de neurona de Wilson. Ejecuta la celda siguiente para ver un gráfico interactivo:

In [ ]:
@widgets.interact(amp=(-1.5, 1.5, 0.1), tau_R=(1.5, 4.5, 0.01), g_K=(15, 30, 0.01), g_T=(0, 2.5, 0.01), g_H=(0, 10, 0.01))
def simulate(amp=1, tau_R=4.5, g_K=26, g_T=2.25, g_H=9.5):
  t = np.arange(0, 200, 0.05)
  I = I_inj(t, amp)
  V, R, T, H = integrar_wilson(t, I, tau_R=tau_R, g_K=g_K, g_T=g_T, g_H=g_H)
  plot_wilson(t, I, V, R, T, H)

Se les puede asignar diferentes valores a lo parámetros $\tau_R$, $g_T$ y $g_H$ para simular distintas formas de comportamiento dinámico observado en neuronas corticales.

### Neuronas RS

Veamos que sucede cuando inyectamos una corriente constante de $I_{iny}=1mA$ durante $200{ms}$ usando los siguientes parámetros:

$\begin{split} \tau_R &= 1.5 \\ g_T &= 2.25 \\ g_H &= 0 \end{split}$

In [ ]:
amp = 1.0
t = np.arange(0, 200, 0.05)
I = I_inj(t, amp)
V, R, T, H = integrar_wilson(t, I, tau_R=4.2, g_T=0.1, g_H=5)
plot_wilson(t, I, V, R, T, H)

### Neuronas FS

Algunas neuronas corticales inhibitorias exhiben un característico disparo rápido. Estas se pueden simular usando:

$\begin{split} \tau_R &= 1.5 \\ g_T &= 0.25 \\ g_H &= 0 \end{split}$

In [ ]:
amp = 1.0
t = np.arange(0, 200, 0.05)
I = I_inj(t, amp)
V, R, T, H = integrar_wilson(t, I, tau_R=1.5, g_T=0.25, g_H=0)
plot_wilson(t, I, V, R, T, H)

### Neuronas CB

Se han reportado ráfagas contínuas de disparos en respuesta a una estimulación constante en algunas neuronas corticales. Para simular estas neuronas, se pueden usar los siguientes valores:

$\begin{split} \tau_R &= 4.2 \\ g_T &= 2.25 \\ g_H &= 9.5 \end{split}$

In [ ]:
amp = 1.0
t = np.arange(0, 200, 0.05)
I = I_inj(t, amp)
V, R, T, H = integrar_wilson(t, I, tau_R=4.2, g_T=2.25, g_H=9.5)
plot_wilson(t, I, V, R, T, H)

### Neuronas  IB

Las neuronas excitatorias IB exhiben una ráfaga inicial, seguido de una pausa y luego comienzan a disparar en forma regular. Para simular estas neuronas, se pueden usar los siguientes valores:

$\begin{split} \tau_R &= 4.2 \\ g_T &= 0.8 \\ g_H &= 4.0 \end{split}$

In [ ]:
amp = 1.0
t = np.arange(0, 200, 0.05)
I = I_inj(t, amp)
V, R, T, H = integrar_wilson(t, I, tau_R=4.2, g_T=0.8, g_H=4.0)
plot_wilson(t, I, V, R, T, H)

## Referencias

Wilson, H. R. (1999). Simplified Dynamics of Human and Mammalian Neocortical Neurons. *Journal of Theoretical Biology*, 200(4), 375–388. [doi:10.1006/jtbi.1999.1002](https://doi.org/10.1006/jtbi.1999.1002)